In [1]:
import os
import glob
import psycopg2 as pg 
import pandas as pd 
from queries import *
import numpy as np

In [2]:
def process_song_file(cur, filepath):
    song_df = pd.read_json(filepath, lines=True)

    for value in song_df.values:
        num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = value

        # insert artist record
        artist_data = [artist_id, artist_name, artist_location, artist_longitude, artist_latitude]

        cur.execute(artist_table_insert, artist_data)

        # insert song record
        song_data = [song_id, title, artist_id, year, duration]
        cur.execute(song_table_insert, song_data) 

In [3]:
def process_log_file(cur, filepath):
    log_df = pd.read_json(filepath, lines=True)
    log_df.replace('', np.nan, inplace=True)
    log_df.dropna(subset=['userId'], inplace=True)
    time_df = log_df[log_df['page']=='NextSong']

    # convert timestamp column to datetime
    time = pd.to_datetime(time_df['ts'], unit='ms')
    

    # insert time data records
    time_data = []

    for line in time:
        time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.day_name()])
        
    column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
    time_df = pd.DataFrame.from_records(time_data, column_labels)


    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))


    # load user table
    user_df = log_clean_df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in log_clean_df.iterrows():
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()

        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)


        songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)

        cur.execute(songplay_table_insert, songplay_data)


In [4]:
def process_data(cur, conn, filepath, func):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    for i, data_file in enumerate(all_files, 1):
        func(cur, data_file)
        conn.commit()
        print("{} / {} files processed".format(i, num_files))

    return all_files

In [5]:
conn = pg.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

process_data(cur, conn, filepath='data/song_data', func=process_song_file)

conn.close()

71 files found in data/song_data
1 / 71 files processed
2 / 71 files processed
3 / 71 files processed
4 / 71 files processed
5 / 71 files processed
6 / 71 files processed
7 / 71 files processed
8 / 71 files processed
9 / 71 files processed
10 / 71 files processed
11 / 71 files processed
12 / 71 files processed
13 / 71 files processed
14 / 71 files processed
15 / 71 files processed
16 / 71 files processed
17 / 71 files processed
18 / 71 files processed
19 / 71 files processed
20 / 71 files processed
21 / 71 files processed
22 / 71 files processed
23 / 71 files processed
24 / 71 files processed
25 / 71 files processed
26 / 71 files processed
27 / 71 files processed
28 / 71 files processed
29 / 71 files processed
30 / 71 files processed
31 / 71 files processed
32 / 71 files processed
33 / 71 files processed
34 / 71 files processed
35 / 71 files processed
36 / 71 files processed
37 / 71 files processed
38 / 71 files processed
39 / 71 files processed
40 / 71 files processed
41 / 71 files pr

In [6]:
conn = pg.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

process_data(cur, conn, filepath='data/log_data', func=process_log_file)

conn.close()

30 files found in data/log_data


ValueError: Shape of passed values is (73, 7), indices imply (7, 7)

In [1]:
cur.close()
cur.closed

NameError: name 'cur' is not defined